# PyQAlloy Basic Usage Examples (over the ULTERA Database)

**Welcome to this minimal Jupyter notebook tutorial that shows how to use the PyQAlloy package functions over an established dataset following the expected schema (of the ULTERA Database). It showcases PyQAlloy's deployment over an older snapshot (`CURATED_Dec2022` collection in `ULTERA_internal` sub-database) to curate the dataset with all core methods:**
- [Single Composition Analyzer](#Single-Composition-Analyzer) which operates within scope of a single composition, i.e., does not require any study reference, property data, or other compositions from the database.
- [Single DOI Analyzer](#Single-DOI-Analyzer) which operates within scope of a single study / reference, which should ideally be a DOI, putting many compositions and properties together.
- [All Data (Entire Database) Analyzer](#All-Data-(Entire-Database)-Analyzer) which operates within the scope of the entire database, leveraging multi-study patterns and relationships to curate the data.

**Please note that to execute run this notebook, you need to have API credentials either stored in `credentials.json` file, set using `pyqalloy.setCredentials`, or set using `pyqalloy.setCredentialsFromURI`. If you don't have them, you can still follow it along looking at the pre-computed results to learn the usage of `PyQAlloy` over static and explained results.** Tutorials 2 and 3 will later show you how to use a static snapshot of a database (in-memory) and how to use the PyQAlloy package to curate your data conforming to the ULTERA schema standards.

Let's start by:
- Setting up the ULTERA API credentials. 
- Importing the `analysis` submodule from `pyqalloy`.

In [ ]:
from pyqalloy import setCredentials

# Once again, if you don't have them, you can still follow along looking at the pre-computed results in this notebook - just don't run the cells.
setCredentials(
    name='myname',
    dbKey='mydbkey',
    dataServer='mycluster.h20x21.mongodb.net'
)

In [1]:
# All relevant classes for this tutorial are in the analysis submodule
from pyqalloy.curation import analysis

## Single Composition Analyzer

Mehtods of the `SingleCompositionAnalyzer` class explicitly operate within the scope of a single composition, i.e., interpretation of a chemical formula stored in the database and then parsed into several formats (raw, fractional, percentile, relational) defined in ULTERA. 

In the context of compositionally complex materials (CCMs), like multi-principle element alloys (MPEAs), high-entropy alloys (HEAs), high-entropy ceramics (HECs), Ni-based superalloys, or advanced steels, the most powerful (yet simple) method is the `scanCompositionsAround100` method which, as the name suggests, scans looking for compositions that sum to around `100%` (or `1` in fractional form), yet not exactly `100%`; thus, suggesting either (a) parsing errors or (b) normalization errors.

Start by setting up a `SingleCompositionAnalyzer` object called, e.g. `sC`. By default, it will connect to the `'ULTERA_internal'` database's `'CURATED_Dec2022'` collection we use for demonstration purposes in PyQAlloy as of `v0.4.0`.

In [2]:
sC = analysis.SingleCompositionAnalyzer()

Loading the database credentials from default location: /Users/adam/VSCode Projects/PyQAlloy-1/pyqalloy/credentials.json
Connected to the CURATED_Dec2022 in ULTERA_internal with 6073 data points detected.


However, you can easily point it to another database, by pointing it to a MongoDB-compatible Collection object or custom server by providing custom `collection`, `database`, and `credentialsFile` arguments, as shown in the commented-out line below.

In [3]:
# sC = analysis.SingleCompositionAnalyzer(collectionManualOverride=my_mongo_collection_obejct)
# sC = analysis.SingleCompositionAnalyzer(database='mydatabase', collection='mycollection', credentialsFile='../mycredentials.json')

Once connected, the use is quite straightforward and the default values tend to work well for most cases. Start by scanning through all the compositions with `scanCompositionsAround100`, described earlier, looking for the the fraction sums that are close to `100%` but not exactly `100%`. To keep things simple, you can request to limit the results up to `10` results and print them to the console on the fly, so you that you can just visually inspect them as you go.

In [4]:
sC.scanCompositionsAround100(resultLimit=10, printOnFly=True)

DOI: 10.1016/j.msea.2017.04.111
F:   Cr19 Fe19 Co19 Ni37 Cu4 Al4
PF:  Cr18.6 Fe18.6 Co18.6 Ni36.3 Cu3.9 Al3.9
Raw:  Al4Co19Cr19Cu4Fe19Ni37
RF:  Cr4.75 Fe4.75 Co4.75 Ni9.25 Cu1 Al1
[19.0, 19.0, 19.0, 37.0, 4.0, 4.0]
-->  102.0

DOI: 10.3390/ma12071136
F:   Li38 Ca1 Mg48 Al15 Si1
PF:  Li36.9 Ca1 Mg46.6 Al14.6 Si1
Raw:  Al15Li38Mg48Ca1 Si1
RF:  Li38 Ca1 Mg48 Al15 Si1
[38.0, 1.0, 48.0, 15.0, 1.0]
-->  103.0

DOI: 10.1016/j.msea.2012.04.067  --> F9
F:   Hf1.4 Zr0.007 Ti0.4 Ta3.3 W9.4 Mo0.5 Cr8.1 Co9.3 Ni61.5 Al5.7 B0.017 C0.07
PF:  Hf1.4 Zr0 Ti0.4 Ta3.3 W9.4 Mo0.5 Cr8.1 Co9.3 Ni61.7 Al5.7 B0 C0.1
Raw:  Ni61.5 W9.4 Co9.3 Cr8.1 Al5.7 Ta3.3 Hf1.4 Ti0.4 Mo0.5 C0.07 B0.017 Zr0.007
RF:  Hf2.8 Zr0.01 Ti0.8 Ta6.6 W18.8 Mo1 Cr16.2 Co18.6 Ni123 Al11.4 B0.03 C0.14
[1.4, 0.007, 0.4, 3.3, 9.4, 0.5, 8.1, 9.3, 61.5, 5.7, 0.017, 0.07]
-->  99.694

DOI: 10.1016/j.ijfatigue.2018.08.029  --> T6
F:   Ti86.2 V3.15 Al10.2
PF:  Ti86.6 V3.2 Al10.2
Raw:  Ti86.2 Al10.2 V3.15
RF:  Ti27.37 V1 Al3.24
[86.2, 3.15, 10.2]

You should now see a number of suspecious compositions! For each of them, we get:
- `DOI` of the study where the composition was found.
- `F` is the interpreted formula created by parsing the chemical formula string provided in the uploaded data.
- `PF` is the percentile formula, where all the elements have been normalized to sum to `100%`.
- `Raw` is the raw formula provided in the uploaded data, from which the `F`/`PF`/`RF` were derived.
- `RF` is the relational formula, where the element ratios are normalized to the lest common one, conveniently expressing relative ratios.

Let's have a quick look at three of them!

1. The fist, coming from `10.1016/j.msea.2017.04.111`, sums to `102%`. A quick comparison between `F` and `PF` shows that the formula has probably been approximated too much, in order to match `1%` quantization, while it was originally designed to be relational. A quick journey to the abstract of the paper confirms that - it was `Al0.2CrFeCoNi2Cu0.2` or `AlCr5Fe5Co5Ni10Cu1` in the relational form. Comparing that to the `RF` of the database entry, we can quickly see that the fractions of Cu and Al are overestimated, while the rest is underestimated, with Ni being underestimated the most.

2. Next one, coming from `10.3390/ma12071136` may seem to follow a similar problem at first, but it is more trivial. A quick look at the abstract reveals that the studies formula was `Al15Li35Mg48Ca1Si1` and someone parsing the publication typed `Li38` instead of `Li35` because the next fraction `Mg48` ended with `8` and human brains are prone to repetition.

3. Lastly, the one from `10.1016/j.msea.2012.04.067` adds up to `99.694`, which tipped of the detection algorithm under the `0.2%` uncertainty threshold, but could be accounted as a rounding error without significant effect on the ratio of the elements in the composition and could be considered as a false-positive.

Now, if you saw a lot of anomalies detected which seem like rounding errors, you can re-initialize the `sC` object and run again with custom settings (e.g., `uncertainty=1`, i.e., `+/-1%` passed as close enough to `100%`). There are quite a few settings you can modify, so feel free to explore the documentation or ask for help if you need it.

In [5]:
sC = analysis.SingleCompositionAnalyzer()
sC.scanCompositionsAround100(resultLimit=10, printOnFly=True, uncertainty=1)

Loading the database credentials from default location: /Users/adam/VSCode Projects/PyQAlloy-1/pyqalloy/credentials.json
Connected to the CURATED_Dec2022 in ULTERA_internal with 6073 data points detected.
DOI: 10.1016/j.msea.2017.04.111
F:   Cr19 Fe19 Co19 Ni37 Cu4 Al4
PF:  Cr18.6 Fe18.6 Co18.6 Ni36.3 Cu3.9 Al3.9
Raw:  Al4Co19Cr19Cu4Fe19Ni37
RF:  Cr4.75 Fe4.75 Co4.75 Ni9.25 Cu1 Al1
[19.0, 19.0, 19.0, 37.0, 4.0, 4.0]
-->  102.0

DOI: 10.3390/ma12071136
F:   Li38 Ca1 Mg48 Al15 Si1
PF:  Li36.9 Ca1 Mg46.6 Al14.6 Si1
Raw:  Al15Li38Mg48Ca1 Si1
RF:  Li38 Ca1 Mg48 Al15 Si1
[38.0, 1.0, 48.0, 15.0, 1.0]
-->  103.0

DOI: 10.1016/j.actamat.2016.11.016
F:   Cr16 Fe16 Co16 Ni34.4 Al16
PF:  Cr16.3 Fe16.3 Co16.3 Ni35 Al16.3
Raw:  Al16Co16Cr16Fe16Ni34.4
RF:  Cr1 Fe1 Co1 Ni2.15 Al1
[16.0, 16.0, 16.0, 34.4, 16.0]
-->  98.4

DOI: Liu_1999_ProcessingAndHighTemperature  --> F2
F:   Hf0.9 Mo91
PF:  Hf1 Mo99
Raw:  Mo91 Hf0.9
RF:  Hf1 Mo101.11
[0.9, 91.0]
-->  91.9

DOI: 10.1134/S2070205120040231  --> T2
F:   

Now, you will more frequently see some "far off" cases that may require deeper interpretation. For instance:

1. The `Ta1 Nb0.103` from `10.1134/S2070205120040231` sums to `1.103` and a quick look at its `RF` of `TaNb0.103` can point us to suspect that it was parsed from a formula like `Ta-Nb10`, where fraction of `Ta` has been implicit (being in fact `0.897`) and was not included when the formula was parsed, thus overestimating the fraction of `Ta`. Were did the `.3` come from then? A quick look at the abstract confirms the conjecture about implicit `Ta` fraction, but complicates things because the formulas were mass-based! To some surprise, if we then run another composition `Ta-Nb5` or `Ta95 N5 (wt%)` through `pymatgen` with `from pymatgen.core import Composition; print(Composition.from_weight_dict({'Ta': 0.95, 'Nb': 0.05}))` we get the `Ta90.7 Nb9.3` percentile formula `PyQAlloy` just presented to us based on the `Ta1 Nb0.103`; hence, the datapoint was actually correct all along because someone went to extended lenghts to adjust an implicit weight fraction based formula to a molar fraction relational formula!

2. Going forward, the same study on `10.1134/S2070205120040231` appears again and one may be quick to dismiss it as another peticiular false-positive but just like in an M. Night Shyamalan's movie, there can be yet another plot twist comming, i.e, unlike in the last case, here the `RF` is integer based, so the conversion from implicit weight fraction based formula to a molar fraction relational formula is unlikely, what is confirmed when numbers are run - in the case of `Ta-W` system, the person parsing the paper forgot to apply the same method and data contains the error we suspected in the first place!

For more granular analysis, especially over larger collaborative databases, you may want to only look at compositions that a specific researcher uploaded by initilizing the `sC` with the `name` field specified. This time the `printOnFly` is set to `False`, so that the results are not printed on the fly, but rather stored inside `sC` object for further analysis.

In [6]:
sC = analysis.SingleCompositionAnalyzer(name='Adam Krajewski')
sC.scanCompositionsAround100(
    printOnFly=False, 
    resultLimit=100, 
    uncertainty=0.21)

Loading the database credentials from default location: /Users/adam/VSCode Projects/PyQAlloy-1/pyqalloy/credentials.json
Connected to the CURATED_Dec2022 in ULTERA_internal with 6073 data points detected.


Now, you can save that list into a file for later analysis!

In [7]:
sC.writeResultsToFile('singleComp_Adam.txt')

Or, you can also see them by accessing the internal `list` of `str` with result printouts. Let's have a look at the first three of them.

In [8]:
sC.printOuts[:3]

['DOI: 10.1016/j.msea.2017.04.111\nF:   Cr19 Fe19 Co19 Ni37 Cu4 Al4\nPF:  Cr18.6 Fe18.6 Co18.6 Ni36.3 Cu3.9 Al3.9\nRaw:  Al4Co19Cr19Cu4Fe19Ni37\nRF:  Cr4.75 Fe4.75 Co4.75 Ni9.25 Cu1 Al1\n[19.0, 19.0, 19.0, 37.0, 4.0, 4.0]\n-->  102.0\n',
 'DOI: 10.3390/ma12071136\nF:   Li38 Ca1 Mg48 Al15 Si1\nPF:  Li36.9 Ca1 Mg46.6 Al14.6 Si1\nRaw:  Al15Li38Mg48Ca1 Si1\nRF:  Li38 Ca1 Mg48 Al15 Si1\n[38.0, 1.0, 48.0, 15.0, 1.0]\n-->  103.0\n',
 'DOI: 10.1016/j.actamat.2016.06.063\nF:   Mo7 Cr23 Fe23 Co23 Ni23\nPF:  Mo7.1 Cr23.2 Fe23.2 Co23.2 Ni23.2\nRaw:  Co23Cr23Fe23Ni23Mo7\nRF:  Mo1 Cr3.29 Fe3.29 Co3.29 Ni3.29\n[7.0, 23.0, 23.0, 23.0, 23.0]\n-->  99.0\n']

## Single DOI Analyzer

The next context in which we can look at the alloy data in a database is the `SingleDOIAnalyzer` class, which operates within the scope of a single study / reference, which should ideally be a DOI, putting many data points together. 

It also enables us to deploy more robust methods when it comes to detecting anomalies, including investigation of the distance between nearest neighbors in the compositional space and investigation of patterns they from once projected into lower dimensional spaces.

Let's start by picking some DOI present in the ULTERA database, like `10.1016/j.jallcom.2008.11.059`, and looking at the data coming from it. To do so, we will initialize the `dDOI` object with the DOI of interest.

In [9]:
from pyqalloy.curation import analysis

In [10]:
doi = '10.1016/j.jallcom.2008.11.059'
sDOI = analysis.SingleDOIAnalyzer(doi=doi)

Loading the database credentials from default location: /Users/adam/VSCode Projects/PyQAlloy-1/pyqalloy/credentials.json
Connected to the CURATED_Dec2022 in ULTERA_internal with 6073 data points detected.
********  Analyzer Initialized  ********


In the later part of the tutorial, scanning all DOI will be covered. To get a list of all of them, you can call `get_allDOIs` function on any `SingleDOIAnalyzer` object (even initialized without the `doi` field) and store this list for later use. 

In [11]:
doiList = sDOI.get_allDOIs()

### NN Distance Analysis

First, let's analyze distances between all the compositions in the specified publication and print them out. 

You will see the L1 distances in the compositional space displayed in the form of (left) absolute value and (right) normalized to maximum value.

In [12]:
sDOI.analyze_nnDistances()
sDOI.print_nnDistances()

--->  10.1016/j.jallcom.2008.11.059
0.0866    |  0.9524     <-- F: Ti0.5 Cr1 Fe1 Co1 Ni1 Cu0.5 Al0.5   | PF: Ti9.1 Cr18.2 Fe18.2 Co18.2 Ni18.2 Cu9.1 Al9.1  | Raw: Al0.5CoCrCu0.5FeNiTi0.5   | RF: Ti1 Cr2 Fe2 Co2 Ni2 Cu1 Al1
0.0909    |  1.0        <-- F: Ti0.5 Cr1 Fe1 Co1 Ni1 Cu0.25 Al0.75 | PF: Ti9.1 Cr18.2 Fe18.2 Co18.2 Ni18.2 Cu4.5 Al13.6 | Raw: Al0.75CoCrCu0.25FeNiTi0.5 | RF: Ti2 Cr4 Fe4 Co4 Ni4 Cu1 Al3
0.0823    |  0.9048     <-- F: Ti0.5 Cr1 Fe1 Co1 Ni1 Cu0.75 Al0.25 | PF: Ti9.1 Cr18.2 Fe18.2 Co18.2 Ni18.2 Cu13.6 Al4.5 | Raw: Al0.25CoCrCu0.75FeNiTi0.5 | RF: Ti2 Cr4 Fe4 Co4 Ni4 Cu3 Al1
0.0909    |  1.0        <-- F: Ti0.5 Cr1 Fe1 Co1 Ni1 Cu1           | PF: Ti9.1 Cr18.2 Fe18.2 Co18.2 Ni18.2 Cu18.2       | Raw: CoCrCuFeNiTi0.5           | RF: Ti1 Cr2 Fe2 Co2 Ni2 Cu2    
0.0823    |  0.9048     <-- F: Ti0.5 Cr1 Fe1 Co1 Ni1 Cu0.5 Al0.25  | PF: Ti9.5 Cr19 Fe19 Co19 Ni19 Cu9.5 Al4.8          | Raw: Al0.25CoCrCu0.5FeNiTi0.5  | RF: Ti2 Cr4 Fe4 Co4 Ni4 Cu2 Al1




The left one is primarily useful for understanding if the distances follow expectations, i.e., are not too low (e.g. 0.001%) or too high, while the right distance metric is useful for gauging the consistency of the data, i.e., is it roughly uniformly spaced or some point appear to be missing or out-of-order far.

In the case of the first DOI we looked at, the `'10.1016/j.jallcom.2008.11.059'`, we see that all of them are consistently spaced by around `9%`, what matches our

You can also set the name of the researcher to get the same results, but only if they have contributed to the data reported for the study of interest, i.e., they are points of contact if the data appears to be abnormal or needs corrections.

In [15]:
sDOI.setName('Zi-Kui')

In [16]:
sDOI.analyze_nnDistances()
sDOI.print_nnDistances()

Skipping 10.1016/j.jallcom.2008.11.059. Specified researcher (Zi-Kui) not present in the group ({'Adam Krajewski'})



If you are iterating over many DOI, you may not want to see all the verbose output if there is nothing to analyze. In that case, you can set the `skipFailed=True` to mute the output.

In [17]:
sDOI.analyze_nnDistances()
sDOI.print_nnDistances(skipFailed=True)

We can now set the name to match any of the researchers by setting it to `None` and iterate over the list of all DOIs to get the distances present in each individual study. Let's go over 20 studies between `50`th and `70`th DOI in the database snapshot, which were tested before preparing this tutorial and are known to contain some interesting patterns.

To skip over some of the DOIs for which the data is less likely to be anomalous, we can apply a couple additional options that filter for certain patterns. For instance, `skipNearEquidistant` let's us focus on studies in which the spacing was not uniform (distance is over `nearEquidistantThreshold` fraction of the maximum one for all).

In [18]:
sDOI.setName(None)
for doi in doiList[50:70]:
    sDOI.setDOI(doi)
    sDOI.analyze_nnDistances()
    sDOI.print_nnDistances(
        minSamples=3,
        skipNearEquidistant=True,
        nearEquidistantThreshold=0.5,
        skipFailed=True)

--->  10.3390/ma14071660
0.0488    |  0.078      <-- F: W1 Fe2.1 Ni4.9       | PF: W12.5 Fe26.2 Ni61.3       | Raw: WNi4.9Fe2.1      | RF: W1 Fe2.1 Ni4.9         
0.5392    |  0.8627     <-- F: W1 Re6 Fe2 Ni8       | PF: W5.9 Re35.3 Fe11.8 Ni47.1 | Raw: WNi8Fe2Re6       | RF: W1 Re6 Fe2 Ni8         
0.625     |  1.0        <-- F: Ta5 W1 Fe3 Ni7       | PF: Ta31.2 W6.2 Fe18.8 Ni43.8 | Raw: WNi7Fe3Ta5       | RF: Ta5 W1 Fe3 Ni7         
0.0714    |  0.1143     <-- F: W1 Re1 Fe3 Ni7       | PF: W8.3 Re8.3 Fe25 Ni58.3    | Raw: WNi7Fe3Re        | RF: W1 Re1 Fe3 Ni7         
0.0682    |  0.1091     <-- F: W1 Fe3 Ni7           | PF: W9.1 Fe27.3 Ni63.6        | Raw: WNi7Fe3          | RF: W1 Fe3 Ni7             
0.0465    |  0.0743     <-- F: W1 Re0.2 Fe2.1 Ni4.9 | PF: W12.2 Re2.4 Fe25.6 Ni59.8 | Raw: WNi4.9Fe2.1Re0.2 | RF: W5 Re1 Fe10.5 Ni24.5   
0.0465    |  0.0743     <-- F: W1 Re0.4 Fe2.1 Ni4.9 | PF: W11.9 Re4.8 Fe25 Ni58.3   | Raw: WNi4.9Fe2.1Re0.4 | RF: W2.5 Re1 Fe5.25 Ni12.25


--->  1

While the method is capturing quite many correct compositions and requires careful manual inspection by an expert to detect anomalies, it can quicky capture some patterns that may be missed otherwise. The first study we get to look at, `10.3390/ma14071660`, may seem to have ordinary at first, but a close inspection reveals that two of the alloys are quite dissimilar in a consistent fashion while their `Raw` formulas are oddly similar. A quick look into the methods section of the manuscript quickly reveals that the alloys were reported using notation like `W–8Ni–2Fe–6Re` and `W–4.9Ni–2.1Fe` which got incorrectly parsed as `W8Ni2Fe6Re` and `W4.9Ni2.1Fe` and the two events of change in the minor alloying elements present represented major unwarranted compositional change.

It is worth noting, however, that such cases of much higher errors can also be attributed to many other patterns, such as authors reporting on one or two alloys used as a reference in the study, such as is the case in the next report from `10.1016/j.ijrmhm.2020.105451` being detected by the method.

### Projection Pattern Analysis

In [10]:
doi = '10.1016/j.jallcom.2008.11.059'
sDOI = analysis.SingleDOIAnalyzer(doi=doi)

Loading the database credentials from default location: /Users/adam/VSCode Projects/PyQAlloy-1/pyqalloy/credentials.json
Connected to the CURATED_Dec2022 in ULTERA_internal with 6073 data points detected.
********  Analyzer Initialized  ********


In [11]:
sDOI.get_compVecs_2DPCA()
sDOI.analyze_compVecs_2DPCA()

In [11]:
for doi in ['10.1016/j.jallcom.2008.11.059', '10.3390/met9010076', '10.1016/j.scriptamat.2018.10.023', '10.1007/978-1-4684-6066-7', '10.3390/e18050189']:
    sDOI.setDOI(doi)
    sDOI.getCompVecs()
    if len(sDOI.compVecs)>1:
        sDOI.get_compVecs_2DPCA()
        sDOI.analyze_compVecs_2DPCA(showFigure=True)

In [18]:
for doi in doiList[0:10]:
    sDOI.setDOI(doi)
    sDOI.getCompVecs()
    if len(sDOI.compVecs) > 1:
        sDOI.get_compVecs_2DPCA()
        sDOI.analyze_compVecs_2DPCA(showFigure=True)

In [19]:
sDOI = analysis.SingleDOIAnalyzer(doi='', name='Adam Krajewski')
for doi in doiList[0:10]:
    sDOI.setName('Adam Krajewski')
    sDOI.setDOI(doi)
    sDOI.getCompVecs()
    if len(sDOI.compVecs) > 1:
        sDOI.get_compVecs_2DPCA()
        sDOI.analyze_compVecs_2DPCA(showFigure=True)

Loading the database credentials from default location: /Users/adam/VSCode Projects/PyQAlloy-1/pyqalloy/credentials.json
Connected to the CURATED_Dec2022 in ULTERA_internal with 6073 data points detected.
********  Analyzer Initialized  ********


In [20]:
doi = '10.1016/j.jallcom.2008.11.059'
sDOI = analysis.SingleDOIAnalyzer(doi=doi)
sDOI.setName('Hui Sun')
doiList = sDOI.get_allDOIs()
print(f'{len(doiList)} DOIs to process')

Loading the database credentials from default location: /Users/adam/VSCode Projects/PyQAlloy-1/pyqalloy/credentials.json
Connected to the CURATED_Dec2022 in ULTERA_internal with 6073 data points detected.
********  Analyzer Initialized  ********
479 DOIs to process


In [21]:
toPrintList = []
for doi in doiList:
    sDOI.setName('Hui Sun')
    sDOI.setDOI(doi)
    sDOI.getCompVecs()
    if len(sDOI.compVecs)>1:
        sDOI.get_compVecs_2DPCA()
        out = sDOI.analyze_compVecs_2DPCA(showFigure=False)
        toPrintList.append(out)
    else:
        toPrintList.append(f'Skipping {doi:<30} Not enough data for PCA (N>=2).')
sDOI.writeManyPlots(toPlotList=toPrintList, workbookPath='SingleDOI_ResultPCA_Hui.xlsx')

### All Data (Entire Database) Analyzer 

Set up the allD (all Data) Analyzer Object and initialize it

In [22]:
from pyqalloy.curation import analysis

In [23]:
allD = analysis.AllDataAnalyzer()

Loading the database credentials from default location: /Users/adam/VSCode Projects/PyQAlloy-1/pyqalloy/credentials.json
Connected to the CURATED_Dec2022 in ULTERA_internal with 6073 data points detected.
Updating the list of all unique composition points...
Number of unique formulas found: 1311
Elements Found: {'Li', 'Ga', 'Hf', 'Al', 'N', 'Fe', 'Co', 'Mn', 'Ru', 'Zr', 'Nb', 'Sn', 'C', 'Y', 'Ir', 'Si', 'Ti', 'Cr', 'S', 'Ge', 'W', 'B', 'Ca', 'Mg', 'O', 'Sc', 'Nd', 'Pd', 'Zn', 'Mo', 'Ag', 'Ta', 'Ni', 'Re', 'Be', 'Cu', 'V'}
Done!


In [24]:
allD.getTSNE(perplexity=5)

array([[ 63.90454  , -32.218098 ],
       [ 13.699746 , -11.2030325],
       [ 40.180195 ,  47.422024 ],
       ...,
       [  2.8812213, -55.484604 ],
       [  3.2550993, -55.57655  ],
       [-77.478714 , -18.556082 ]], dtype=float32)

In [25]:
allD.showTSNE()

In [26]:
allD.getDBSCAN(eps=0.075, min_samples=2)

Found 115 clusters and 427 outliers.
Outlier ratio: 32.6%


(array([-1,  0,  1, ..., -1, -1, 20]), 427)

In [27]:
allD.showClustersDBSCAN()

In [28]:
allD.showOutliersDBSCAN()

In [29]:
allD.getDBSCANautoEpsilon(outlierTargetN=17)

Running DBSCAN with eps=1.0...
Found 1 clusters and 0 outliers.
Outlier ratio: 0.0%
Running DBSCAN with eps=0.975...
Found 1 clusters and 0 outliers.
Outlier ratio: 0.0%
Running DBSCAN with eps=0.95...
Found 1 clusters and 0 outliers.
Outlier ratio: 0.0%
Running DBSCAN with eps=0.925...
Found 1 clusters and 0 outliers.
Outlier ratio: 0.0%
Running DBSCAN with eps=0.9...
Found 1 clusters and 0 outliers.
Outlier ratio: 0.0%
Running DBSCAN with eps=0.875...
Found 1 clusters and 0 outliers.
Outlier ratio: 0.0%
Running DBSCAN with eps=0.85...
Found 1 clusters and 0 outliers.
Outlier ratio: 0.0%
Running DBSCAN with eps=0.825...
Found 1 clusters and 0 outliers.
Outlier ratio: 0.0%
Running DBSCAN with eps=0.8...
Found 1 clusters and 0 outliers.
Outlier ratio: 0.0%
Running DBSCAN with eps=0.775...
Found 1 clusters and 0 outliers.
Outlier ratio: 0.0%
Running DBSCAN with eps=0.75...
Found 1 clusters and 0 outliers.
Outlier ratio: 0.0%
Running DBSCAN with eps=0.725...
Found 1 clusters and 0 outlier

(array([0, 1, 0, ..., 0, 0, 0]), 17)

In [30]:
allD.showOutliersDBSCAN()

In [31]:
allD.updateOutliersList()

In [32]:
allD.findOutlierDataSources();

Outlier Zr1 Ta2.13 Ru3.53         | Zr15 Ta32 Ru53            | Ru53 Ta32 Zr15 
matched to:  Marcia Ahn           upload from DOI 10.1016/j.apsusc.2015.06.144 at position T3 

Outlier Hf3 Mo1 B14 Si10          | Hf10.7 Mo3.6 B50 Si35.7   | MoSi10B14Hf3
matched to:  Shuang Lin           upload from DOI 10.1016/j.msea.2011.11.001 at position T1 

Outlier Hf1 Zr1 Ti1 Ta1 C4        | Hf12.5 Zr12.5 Ti12.5 Ta12.5 C50 | (Zr0.25Hf0.25Ta0.25Ti0.25)C
matched to:  Hui Sun              upload from DOI 10.1007/s11661-020-06034-2 at position P2 

Outlier Nb8 W2.33 Cr1 Co11 Al11   | Nb24 W7 Cr3 Co33 Al33     | Co33W07Al33Nb24Cr03
matched to:  Adam Krajewski       upload from DOI 10.1038/s41467-019-10533-1 

Outlier Zr1.27 Ta1.07 Ru1         | Zr38 Ta32 Ru30            | Zr38 Ta32 Ru30
matched to:  Marcia Ahn           upload from DOI 10.1016/j.apsusc.2015.06.144 at position T3 

Outlier Ti4.5 Ta6 Nb15 Co1 Ni23.5 | Ti9 Ta12 Nb30 Co2 Ni47    | Ni47Co02Ta12Ti9Nb30
matched to:  Adam Krajewski       uploa